In [8]:
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import os
import wfdb

from collections import Counter

## Data Pre-processing

In [9]:
# get all the record names in a list
namelist = []
namelist = namelist + list(range(100,125,1)) + list(range(200,235,1))
for num in [110,120,204,206,211,216,218,224,225,226,227,229]:
    namelist.remove(num)

len(namelist)

48

In [10]:
def get_window (signals, annotation, sec):
    """
    this function gives a sec-seconds window (sec seconds before, sec seconds after the annotation mark)
    of the ECG signals and assign value 1 if it's PVC beat and 0 otherwise.
    parameter: signals: numpy array containing heart beat record values
               annotation: wfdb.annotation object containing heart beat annotations
               sec: positive integer number indicating the half-width of the window
    return: two lists
            siglist: a list of lists of length 360*2*sec 
            annlist: a list containing 1 if PVC beat, 0 otherwise
    """
    siglist = []
    annlist = []
    
    #loop through the annotation.symbol list
    for i in range(len(annotation.symbol)):
        timestamp = annotation.sample[i] #get the timestamp
        
        #test if that timestamp can have sec seconds before and after window
        windowStart = timestamp - sec*annotation.fs
        windowEnd = timestamp + sec*annotation.fs
        if windowStart >= 0 & windowEnd <= len(signals):
            if annotation.symbol[i] == 'V':
                # check if the length of this strip is 360*2*sec
                if len(signals[windowStart:windowEnd,].flatten().tolist()) == 2*sec*annotation.fs:
                    siglist.append(signals[windowStart:windowEnd,].flatten().tolist())
                    annlist.append(1)
                else:
                    continue
            else:
                # check if the length of this strip is 360*2*sec
                if len(signals[windowStart:windowEnd,].flatten().tolist()) == 2*sec*annotation.fs:
                    siglist.append(signals[windowStart:windowEnd,].flatten().tolist())
                    annlist.append(0)
        else:
            continue
    
    return siglist, annlist

In [11]:
# loop through all record to get all the 10-seconds window signal list and annotation list
# this could be the training dataset for the neural network model
# this takes several minutes to run

ECG_signals = []
PVC_annotations = []

for record in namelist:
    signals, fields = wfdb.rdsamp(str(record), sampfrom=0, sampto='end', channels=[1], pb_dir='mitdb')
    annotation = wfdb.rdann(str(record), 'atr', sampfrom=0, sampto=None, shift_samps=True, pb_dir='mitdb')
    
    signal_list, annotation_list = get_window(signals, annotation, 5)
    
    ECG_signals = ECG_signals + signal_list
    PVC_annotations = PVC_annotations + annotation_list
    
    print('record', record, 'is done.')

record 100 is done.
record 101 is done.
record 102 is done.
record 103 is done.
record 104 is done.
record 105 is done.
record 106 is done.
record 107 is done.
record 108 is done.
record 109 is done.
record 111 is done.
record 112 is done.
record 113 is done.
record 114 is done.
record 115 is done.
record 116 is done.
record 117 is done.
record 118 is done.
record 119 is done.
record 121 is done.
record 122 is done.
record 123 is done.
record 124 is done.
record 200 is done.
record 201 is done.
record 202 is done.
record 203 is done.
record 205 is done.
record 207 is done.
record 208 is done.
record 209 is done.
record 210 is done.
record 212 is done.
record 213 is done.
record 214 is done.
record 215 is done.
record 217 is done.
record 219 is done.
record 220 is done.
record 221 is done.
record 222 is done.
record 223 is done.
record 228 is done.
record 230 is done.
record 231 is done.
record 232 is done.
record 233 is done.
record 234 is done.


In [13]:
len(ECG_signals)

111985

In [14]:
len(PVC_annotations)

111985

In [15]:
# there are 7112 PVC 10-seconds window, 105186 non-PVC 10-seconds window
Counter(PVC_annotations)

Counter({0: 104886, 1: 7099})

In [25]:
# get the index in PVC_annotations for normal beats

normal_index = [i for i in range(len(PVC_annotations)) if PVC_annotations[i]==0]
len(normal_index)

104886

In [21]:
# get the index in PVC_annotations for PVC beats

PVC_index = [i for i in range(len(PVC_annotations)) if PVC_annotations[i]==1]
len(PVC_index)

7099

In [30]:
# randomly select 7099 normal beats

import random

random.seed(77)
small_normal_index = random.sample(normal_index, 7099)

In [31]:
len(small_normal_index)

7099

In [32]:
small_normal_index[:10]

[111839, 33939, 44009, 26542, 32244, 25993, 15648, 39353, 66405, 77501]

In [33]:
# concatenate 7099 normal beats index with 7099 PVC beats index
# as small data index

small_data_index = small_normal_index + PVC_index
len(small_data_index)

14198

In [34]:
# generate samll dataset

small_signals = []
for i in small_data_index:
    small_signals.append(ECG_signals[i])
    
small_annotations = []
for i in small_data_index:
    small_annotations.append(PVC_annotations[i])

In [35]:
len(small_signals)

14198

In [36]:
len(small_annotations)

14198

In [37]:
# write ECG signals data to csv file

import csv

with open('MIT-BIH-small.csv', 'w') as f:
    f_csv = csv.writer(f)
    f_csv.writerows(small_signals)

In [38]:
# write the PVC annotation data to csv file

with open('MIT-BIH-Annotation-small.csv', 'w') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(small_annotations)

## Classification Models

In [39]:
# transform data into numpy array

X = np.array(small_signals)
y = np.array(small_annotations)

In [41]:
X.shape

(14198, 3600)

In [42]:
y.shape

(14198,)

In [44]:
# split train and test set

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

In [46]:
X_train.shape

(9938, 3600)

In [47]:
y_train.shape

(9938,)

In [48]:
X_test.shape

(4260, 3600)

In [49]:
y_test.shape

(4260,)

In [50]:
# the ratio of PVC beats in training set and test set

Counter(y_train)

Counter({0: 4983, 1: 4955})

In [51]:
Counter(y_test)

Counter({0: 2116, 1: 2144})

In [52]:
# baseline model accuracy

2144/(2116+2144)

0.5032863849765258

### Logistic Regression Model

In [53]:
from sklearn.linear_model import LogisticRegression

In [57]:
from sklearn.metrics import confusion_matrix

In [54]:
# only take 2 minutes to run

log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [55]:
# predict

y_train_pred = log_reg.predict(X_train)

In [58]:
# confusion matrix

confusion_matrix(y_train, y_train_pred)

array([[4223,  760],
       [ 885, 4070]], dtype=int64)

In [59]:
# accuracy

(4223+4070)/(4223+760+885+4070)

0.8344737371704568

In [60]:
# True Positive Rate

4070/(885+4070)

0.8213925327951564

In [61]:
# False Positive Rate

760/(4223+760)

0.1525185631145896

In [65]:
# testing result

y_test_pred = log_reg.predict(X_test)
confusion_matrix(y_test, y_test_pred)

array([[1619,  497],
       [ 525, 1619]], dtype=int64)

In [66]:
# accuracy

(1619+1619)/(1619+497+525+1619)

0.760093896713615

In [67]:
# True Positive Rate

1619/(525+1619)

0.7551305970149254

In [68]:
# False Positive Rate

497/(1619+497)

0.23487712665406427

In [62]:
# save the model

import pickle

# Dump the trained logistic regression model with Pickle
LogReg_pkl_filename = 'LogRegSmall.pkl'

# Open the file to save as pkl file
LogReg_model_pkl = open(LogReg_pkl_filename, 'wb')
pickle.dump(log_reg, LogReg_model_pkl)

# Close the pickle instances
LogReg_model_pkl.close()

### Random Forest Model

In [63]:
from sklearn.ensemble import RandomForestRegressor

In [64]:
# only takes 7 minutes to run

forest_reg = RandomForestRegressor()
forest_reg.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [70]:
# training set result

y_train_pred = forest_reg.predict(X_train)
y_train_pred = [int(x) for x in y_train_pred]
confusion_matrix(y_train, y_train_pred)

array([[4983,    0],
       [1396, 3559]], dtype=int64)

In [71]:
# accuracy

(4983+3559)/(4983+1396+3559)

0.8595290802978467

In [72]:
# True Positive Rate

3559/(1396+3559)

0.7182643794147326

In [73]:
# False Positive Rate

0

0

In [74]:
# testing set result

y_test_pred = forest_reg.predict(X_test)
y_test_pred = [int(x) for x in y_test_pred]
confusion_matrix(y_test, y_test_pred)

array([[2106,   10],
       [ 933, 1211]], dtype=int64)

In [75]:
# accuracy

(2106+1211)/(2106+10+933+1211)

0.7786384976525822

In [76]:
# True Positive Rate

1211/(933+1211)

0.5648320895522388

In [77]:
# False Positive Rate

10/(2106+10)

0.004725897920604915

In [78]:
# save the model

import pickle

# Dump the trained random forest classifier with Pickle
RandomForest_pkl_filename = 'RandomForestSmall.pkl'

# Open the file to save as pkl file
RandomForest_model_pkl = open(RandomForest_pkl_filename, 'wb')
pickle.dump(forest_reg, RandomForest_model_pkl)

# Close the pickle instances
RandomForest_model_pkl.close()

### Neural Network Model

In [80]:
import tensorflow as tf

In [81]:
n_inputs = 3600
n_hidden1 = 300
n_hidden2 = 30
n_outputs = 2

In [82]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
y = tf.placeholder(tf.int64, shape=(None), name='y')

In [83]:
# construct neural network

with tf.name_scope('dnn'):
    hidden1 = tf.layers.dense(X, n_hidden1, name='hidden1', activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name='hidden2', activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name='outputs')

In [84]:
# define cost function

with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name='loss')

In [85]:
# define optimizer

learning_rate = 0.01

with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [86]:
# define evaluation

with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [87]:
# create init and saver

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [90]:
# execution

n_epochs = 30
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(len(X_train)//batch_size):
            batch_index = random.sample(range(len(X_train)), batch_size)
            X_batch = X_train[batch_index]
            y_batch = y_train[batch_index]
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
        acc_train = accuracy.eval(feed_dict={X:X_batch, y:y_batch})
        acc_test = accuracy.eval(feed_dict={X:X_test, y:y_test})
        print(epoch, 'Train accuracy:', acc_train, 'Test accuracy:', acc_test)
        
    save_path = saver.save(sess, './dnn_model.ckpt')

0 Train accuracy: 0.86 Test accuracy: 0.797183
1 Train accuracy: 0.88 Test accuracy: 0.834038
2 Train accuracy: 0.88 Test accuracy: 0.854695
3 Train accuracy: 0.92 Test accuracy: 0.870657
4 Train accuracy: 0.94 Test accuracy: 0.878873
5 Train accuracy: 0.96 Test accuracy: 0.881925
6 Train accuracy: 0.98 Test accuracy: 0.884038
7 Train accuracy: 0.98 Test accuracy: 0.888732
8 Train accuracy: 0.94 Test accuracy: 0.892254
9 Train accuracy: 1.0 Test accuracy: 0.89507
10 Train accuracy: 0.96 Test accuracy: 0.89554
11 Train accuracy: 0.94 Test accuracy: 0.901643
12 Train accuracy: 0.94 Test accuracy: 0.902817
13 Train accuracy: 1.0 Test accuracy: 0.902113
14 Train accuracy: 1.0 Test accuracy: 0.902582
15 Train accuracy: 0.98 Test accuracy: 0.906338
16 Train accuracy: 0.98 Test accuracy: 0.909155
17 Train accuracy: 0.96 Test accuracy: 0.908451
18 Train accuracy: 0.98 Test accuracy: 0.91338
19 Train accuracy: 0.98 Test accuracy: 0.912207
20 Train accuracy: 0.98 Test accuracy: 0.917371
21 Train

In [91]:
with tf.Session() as sess:
    saver.restore(sess, './dnn_model.ckpt')
    Z = logits.eval(feed_dict={X:X_test})
    y_pred = np.argmax(Z, axis=1)

INFO:tensorflow:Restoring parameters from ./dnn_model.ckpt


In [92]:
confusion_matrix(y_test, y_pred)

array([[1987,  129],
       [ 202, 1942]], dtype=int64)

In [93]:
# accuracy

(1987+1942)/(1987+129+202+1942)

0.922300469483568

In [94]:
# True Positive Rate

1942/(202+1942)

0.9057835820895522

In [95]:
# False Positive Rate

129/(1987+129)

0.0609640831758034

## Compare Results

|model|accuracy|true positive rate|false positive rate|
|-----|--------|------------------|-------------------|
|baseline|0.5033|0.0|0.0|
|Logistic Regression training|0.8345|0.8214|0.1525|
|Logistic Regression testing|0.7601|0.7551|0.2349|
|Random Forest training|0.8595|0.7183|0.0|
|Random Forest testing|0.7786|0.5648|0.0047|
|Neural Network training|1.0|||
|Neural Network testing|0.9223|0.9056|0.0610|